Import Libraries

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG19, ResNet50
from tensorflow.keras.datasets import cifar10, cifar100
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import pandas as pd

Load Custom Model From Filesystem

In [2]:
custom_model = load_model('Exp1_results/best_model/exp1_best_model.h5')
custom_model.summary()
custom_model.trainable = False

c:\Users\ashna\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 32, 32, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 32, 32, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 32, 32, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 32, 32, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 16, 16, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 16, 16, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 16, 16, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1024)           │     8,389,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │        10,250 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,661,580 (33.04 MB)

 Trainable params: 8,660,810 (33.04 MB)

 Non-trainable params: 768 (3.00 KB)

 Optimizer params: 2 (12.00 B)

Get ImageNet Data

In [3]:
import time
import imageio
import numpy as np
path = 'IMagenet/tiny-imagenet-200/'

def get_id_dictionary():
    id_dict = {}
    for i, line in enumerate(open( path + 'wnids.txt', 'r')):
        id_dict[line.replace('\n', '')] = i
    return id_dict
  
def get_class_to_id_dict():
    id_dict = get_id_dictionary()
    all_classes = {}
    result = {}
    for i, line in enumerate(open( path + 'words.txt', 'r')):
        n_id, word = line.split('\t')[:2]
        all_classes[n_id] = word
    for key, value in id_dict.items():
        result[value] = (key, all_classes[key])      
    return result

def get_imagenet_data(id_dict):
    print('starting loading data')
    train_data, test_data = [], []
    train_labels, test_labels = [], []
    t = time.time()
    for key, value in id_dict.items():
        train_data += [imageio.imread( path + 'train/{}/images/{}_{}.JPEG'.format(key, key, str(i)), mode='RGB') for i in range(500)]
        train_labels_ = np.array([[0]*200]*500)
        train_labels_[:, value] = 1
        train_labels += train_labels_.tolist()

    for line in open( path + 'val/val_annotations.txt'):
        img_name, class_id = line.split('\t')[:2]
        test_data.append(imageio.imread( path + 'val/images/{}'.format(img_name) ,mode='RGB'))
        test_labels_ = np.array([[0]*200])
        test_labels_[0, id_dict[class_id]] = 1
        test_labels += test_labels_.tolist()

    print('finished loading data, in {} seconds'.format(time.time() - t))
    return np.array(train_data), np.array(train_labels), np.array(test_data), np.array(test_labels)

Prepare Datasets

In [4]:

def prepare_data(dataset_name):
    x_train = None
    x_test = None
    y_train = None
    y_test = None
    # CIFAR-100 data preparation
    if dataset_name == 'CIFAR100':
        (x_train, y_train), (x_test, y_test) = cifar100.load_data()
        num_classes = 100
        x_train = x_train.astype('float32') / 255
        x_test = x_test.astype('float32') / 255
        y_train = to_categorical(y_train, num_classes)
        y_test = to_categorical(y_test, num_classes)
    elif dataset_name == 'IMagenet':
        train_data, train_labels, test_data, test_labels = get_imagenet_data(get_id_dictionary())
        x_train = train_data.astype('float32') / 255
        x_test = test_data.astype('float32') / 255
        y_train = train_labels
        y_test = test_labels

    # Resizing images to 32x32
    x_train = tf.image.resize(x_train, (32, 32))
    x_test = tf.image.resize(x_test, (32, 32))

    return (x_train, y_train), (x_test, y_test)


Data Augmentation

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def data_augmentation(x_train):
    datagen = ImageDataGenerator(
        rotation_range=10,
        width_shift_range=0.05,
        height_shift_range=0.05,
        horizontal_flip=True,
        fill_mode='nearest',
        zoom_range=0.1
    )
    datagen.fit(x_train)
    return datagen

Train Model

In [6]:
def train_model(model, train_data, test_data, lr_schedule):
    x_train, y_train = train_data
    x_test, y_test = test_data
    datagen = data_augmentation(x_train)
    model.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_loss', verbose=1, patience=10, restore_best_weights=True)
    history = model.fit(datagen.flow(x_train, y_train, batch_size=64), epochs=100, validation_data=(x_test, y_test), callbacks=[early_stopping, lr_schedule])
    return history

Save Results

In [7]:
def save_results(history, dataset_name):
    results_df = pd.DataFrame(history.history)
    if dataset_name == "CIFAR100":
        base_path = 'Exp2_results/Custom/CIFAR100'
        results_df.to_csv(base_path + 'training_history.csv')
    else:
        base_path = 'Exp2_results/Custom/ImageNet'
        results_df.to_csv(base_path + 'training_history.csv')

        

Define List of Dataset Names

In [8]:
datasets = ['CIFAR100', 'IMageNet']  

Modify Last Layer of Model for different datasets

In [9]:

def modify_model_for_classes(model, num_classes):
    # Explicitly define a new input layer that matches the shape of the original model's input
    if not model.inputs:
        raise ValueError("Model does not have any inputs defined.")
    input_shape = model.inputs[0].shape[1:]  # Get shape, excluding the batch size
    new_input = Input(shape=input_shape)
    
    # Rebuild the model from the new input using all layers except the last
    x = new_input
    for layer in model.layers[:-1]:  # Skip the last layer
        if not layer.__class__.__name__ == 'InputLayer':  # Skip the original input layer
            # Ensure each layer has a unique name by appending a suffix
            layer_config = layer.get_config()
            layer_config['name'] = layer_config['name'] + '_reused'
            new_layer = layer.__class__.from_config(layer_config)
            x = new_layer(x)
    
    # Add a new Dense layer as the output layer with a unique name
    output = Dense(num_classes, activation='softmax', name='output_layer')(x)
    new_model = Model(inputs=new_input, outputs=output)
    return new_model



Main Training Loop

In [10]:
# Implementing a Learning Rate Scheduler
from tensorflow.keras.callbacks import LearningRateScheduler
from keras.optimizers import Adam

def scheduler(epoch, lr):
    if epoch < 10:
        return float(lr)
    else:
        return float(lr * tf.math.exp(-0.1))

lr_schedule = LearningRateScheduler(scheduler)

for dataset in datasets:
    if dataset == 'CIFAR100':
        num_classes = 100
    elif dataset == 'IMageNet':
        num_classes = 200
    model = modify_model_for_classes(custom_model, num_classes)
    train_data, test_data = prepare_data(dataset)
    history = train_model(model, train_data, test_data, lr_schedule)
    save_results(history, dataset)

        
    

starting loading data


C:\Users\ashna\AppData\Local\Temp\ipykernel_48364\3893201637.py:29: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  train_data += [imageio.imread( path + 'train/{}/images/{}_{}.JPEG'.format(key, key, str(i)), mode='RGB') for i in range(500)]
C:\Users\ashna\AppData\Local\Temp\ipykernel_48364\3893201637.py:36: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  test_data.append(imageio.imread( path + 'val/images/{}'.format(img_name) ,mode='RGB'))


finished loading data, in 191.9722740650177 seconds
Epoch 1/100


c:\Users\ashna\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1563/1563 ━━━━━━━━━━━━━━━━━━━━ 496s 314ms/step - accuracy: 0.0069 - loss: 5.7939 - val_accuracy: 0.0051 - val_loss: 5.4091 - learning_rate: 0.0010
Epoch 2/100
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 498s 318ms/step - accuracy: 0.0046 - loss: 5.3696 - val_accuracy: 0.0050 - val_loss: 5.3083 - learning_rate: 0.0010
Epoch 3/100
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 507s 324ms/step - accuracy: 0.0043 - loss: 5.3207 - val_accuracy: 0.0050 - val_loss: 5.5033 - learning_rate: 0.0010
Epoch 4/100
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 531s 340ms/step - accuracy: 0.0042 - loss: 5.4674 - val_accuracy: 0.0050 - val_loss: 5.3847 - learning_rate: 0.0010
Epoch 5/100
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 525s 336ms/step - accuracy: 0.0044 - loss: 5.3700 - val_accuracy: 0.0050 - val_loss: 5.3326 - learning_rate: 0.0010
Epoch 6/100
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 536s 343ms/step - accuracy: 0.0043 - loss: 5.3250 - val_accuracy: 0.0050 - val_loss: 5.3069 - learning_rate: 0.0010
Epoch 7/100
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 596s 381ms/step

KeyboardInterrupt: 